In [ ]:
import os
import sys
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.models as models
import numpy as np
import pandas as pd
from PIL import Image
import sklearn.utils as utils
import sklearn.metrics as metrics
from torchvision import transforms as T
from torchvision.models import resnet50
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import cv2
import pydicom
from sklearn.utils import shuffle
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')

In [ ]:
df

In [ ]:
def agg_min(x):
    if np.isnan(x.values[0]):
        return [0]
    else:
        return list(x)
def agg_max(x):
    if np.isnan(x.values[0]):
        return [1]
    else:
        return list(x)
def agg_class(x):
    if x.values[0] == 14:
        return [0]
    else:
        return list(x.values+1)


In [ ]:
df = df.groupby('image_id').agg({'x_min': agg_min, 'x_max': agg_max, 'y_min': agg_min, 'y_max': agg_max, 'class_id': agg_class})

In [ ]:
df = df.reset_index(drop=False)

In [ ]:
df

Use IOU score to select one of multiple overlapping boxes.

In [ ]:

def iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    if (min(x1_max, x2_max) - max(x1_min, x2_min)) <= 0 or (min(y1_max, y2_max) - max(y1_min, y2_min)) <= 0:
        return 0
    intersection = (min(x1_max, x2_max) - max(x1_min, x2_min)) * (min(y1_max, y2_max) - max(y1_min, y2_min))
    union = (x1_max - x1_min) * (y1_max - y1_min) + (x2_max - x2_min) * (y2_max - y2_min) - intersection
    return intersection / (union + 1e-5)

def get_similarity(x):
    similarities = np.zeros((x.shape[0], x.shape[0]))
    for i in range(x.shape[0]):
        for j in range(i, x.shape[0]):
            similarities[i, j] = iou(x[i], x[j])
    return similarities


In [ ]:
df2 = []
for row_i in range(len(df)):
    row = df.iloc[row_i]
    x = np.array(row[['x_min', 'y_min', 'x_max', 'y_max', 'class_id']].values.tolist()).transpose()
    x_mins, y_mins, x_maxs, y_maxs, class_ids = [], [], [], [], []
    if len(x) > 0:
        similarities = get_similarity(x[:, :-1])
        x_i = x[0]
        x_mins.append(x_i[0])
        y_mins.append(x_i[1])
        x_maxs.append(x_i[2])
        y_maxs.append(x_i[3])
        class_ids.append(x_i[4])
        for i in range(1, len(x)):
            if np.all(similarities[:i, i] < 0.6):
                x_i = x[i]
                x_mins.append(x_i[0])
                y_mins.append(x_i[1])
                x_maxs.append(x_i[2])
                y_maxs.append(x_i[3])
                class_ids.append(x_i[4])
    df2.append([row['image_id'], x_mins, y_mins, x_maxs, y_maxs, class_ids])
    if row_i % 1000 == 0: print(row_i)

df2 = pd.DataFrame(df2, columns=['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'class_id'])

In [ ]:
# df['image_path'] = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/' + df['image_id'] + '.dicom'

In [ ]:
# Dicom Images
# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, df, transforms):
#         self.transforms = transforms
#         self.df = df

#     def __getitem__(self, idx):
#         img_path = self.df.iloc[idx]['image_path']
#         boxes = np.array(self.df.iloc[idx][['x_min', 'y_min', 'x_max', 'y_max']].values.tolist()).transpose()
#         dicom = pydicom.dcmread(img_path)#
#         dicom.BitsStored = 16
#         img = dicom.pixel_array.astype('float32')
#         if dicom.PhotometricInterpretation == "MONOCHROME1":
#             img = np.amax(img) - img
#         intercept = int(dicom.RescaleIntercept) if "RescaleIntercept" in dicom else 0.0
#         slope = float(dicom.RescaleSlope) if "RescaleSlope" in dicom else 1.0
#         img = (img * slope).astype('int16') + intercept
#         img = np.stack([img, img, img])
#         img = (img - img.min()) / (img.max() - img.min()) * 255.
#         img = img.transpose(1, 2, 0)
#         labels = self.df.iloc[idx]['class_id']
#         target = {}
#         boxes = torch.as_tensor(boxes, dtype=torch.float32)
#         if len(boxes) == 0:
#             boxes = torch.tensor([[0., 0., 1., 1.]], dtype=torch.float32)
#             labels = [0]
#         area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
#         target["boxes"] = boxes #/ 1024.
#         target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
#         target["area"] = torch.as_tensor(area, dtype=torch.float32)
#         sample = {
#                     'image': img,
#                     'bboxes': target['boxes'],
#                     'labels': labels
#                 }
#         sample = self.transforms(**sample)
#         img = sample['image']
#         target['boxes'] = torch.tensor(sample['bboxes'])
#         return img, target

#     def __len__(self):
#         return len(self.df)

In [ ]:
df['image_path'] = '/kaggle/input/vinbigdata-train-jpg/' + df['image_id'] + '.jpg'

In [ ]:
# JPG Images
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, transforms):
        self.transforms = transforms
        self.df = df

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        boxes = np.array(self.df.iloc[idx][['x_min', 'y_min', 'x_max', 'y_max']].values.tolist()).transpose()
        img = np.array(Image.open(img_path))
        labels = self.df.iloc[idx]['class_id']
        target = {}
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        if len(boxes) == 0:
            boxes = torch.tensor([[0., 0., 1., 1.]], dtype=torch.float32)
            labels = [0]
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target["boxes"] = boxes #/ 1024.
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        sample = {
                    'image': img,
                    'bboxes': target['boxes'],
                    'labels': labels
                }
        sample = self.transforms(**sample)
        img = sample['image']
        target['boxes'] = torch.tensor(sample['bboxes'])
        return img, target

    def __len__(self):
        return len(self.df)

In [ ]:
df = shuffle(df,random_state=42)
df_train = df[:12000]
df_val = df[12000:]

In [ ]:

train_transforms = A.Compose([
            A.HorizontalFlip(0.5),
            A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
valid_transforms = A.Compose([
            A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
train_dataset = Dataset(df_train, train_transforms)
val_dataset = Dataset(df_val, valid_transforms)

In [ ]:
def collate_fn(batch):
    images = [image for (image, _) in batch]
    targets = [target for (_, target) in batch]
    return images, targets

In [ ]:
train_data_loader = torch.utils.data.DataLoader(
                                         train_dataset,
                                         batch_size=8,
                                         shuffle=True,
                                         collate_fn=collate_fn,
                                         num_workers=2
                                         )
valid_data_loader = torch.utils.data.DataLoader(
                                         val_dataset,
                                         batch_size=8,
                                         shuffle=True,
                                         collate_fn=collate_fn,
                                         num_workers=2
                                         )

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 15
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model = model.to(device)

In [ ]:
opt  = torch.optim.Adam(model.parameters(), lr=0.00001)

In [ ]:
EPOCHS = 0

It is very slow to train in kaggle kernels as the number of CPUs are only 2, GPU remains underutilized I believe. Better to train elsewhere or use lower resolution images.

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
for epoch in range(EPOCHS):
        for b, (x, y) in enumerate(train_data_loader):
            x = [xi.to(device) for xi in x]
            y = [{k: v.to(device) for k, v in yi.items()} for yi in y]
            opt.zero_grad()
            loss_dict = model(x, y)
            loss = loss_dict['loss_box_reg'] + loss_dict['loss_objectness'] + loss_dict['loss_classifier'] + loss_dict['loss_rpn_box_reg']#sum(loss for loss in loss_dict_reduced.values())
            loss.backward()
            opt.step()
            if b % 10 == 0:
                print('Epoch {}/{}, batch {}/{}, loss {:.4f}, loss classifier {:.4f}, loss box reg {:.4f}, loss objectness {:.4f}, loss rpn box reg {:.4f}'\
                      .format(epoch+1, EPOCHS, b+1, len(train_data_loader), loss.item(), loss_dict['loss_classifier'].item(),
                        loss_dict['loss_box_reg'].item(), loss_dict['loss_objectness'].item(), loss_dict['loss_rpn_box_reg'].item()))
            if b % 1000 == 0:
                #model.eval()
                val_loss = {'loss_box_reg': 0, 'loss_classifier': 0, 'loss_objectness': 0, 'loss_rpn_box_reg': 0}
                for _, (x, y) in enumerate(valid_data_loader):
                    x = [xi.to(device) for xi in x]
                    y = [{k: v.to(device) for k, v in yi.items()} for yi in y]
                    loss_dict = model(x, y)
                    loss = loss_dict['loss_box_reg'] + loss_dict['loss_objectness'] + loss_dict['loss_classifier'] + loss_dict['loss_rpn_box_reg']#sum(loss for loss in loss_dict_reduced.values())
                    loss = loss.detach()
                    val_loss = {
                            'loss_box_reg': val_loss['loss_box_reg'] + loss_dict['loss_box_reg'].item(),
                            'loss_classifier': val_loss['loss_classifier'] + loss_dict['loss_classifier'].item(),
                            'loss_objectness': val_loss['loss_objectness'] + loss_dict['loss_objectness'].item(),
                            'loss_rpn_box_reg': val_loss['loss_rpn_box_reg'] + loss_dict['loss_rpn_box_reg'].item(),

                    }

                    val_loss = {loss_key: loss / len(valid_data_loader) for loss_key, loss in val_loss.items()}
                print('loss classifier {:.4f}, loss box reg {:.4f}, loss objectness {:.4f}, loss rpn box reg {:.4f} '\
                      .format(val_loss['loss_classifier'], val_loss['loss_box_reg'], val_loss['loss_objectness'], val_loss['loss_rpn_box_reg']))